In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, RobertaForSequenceClassification
import torch
import numpy as np
from scipy.special import softmax
import json
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model_eng = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
#model.save_pretrained(MODEL)
text = "Hi!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model_eng(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [4]:
ranking = np.argsort(scores)
print(ranking)
ranking = ranking[::-1]
print(ranking)
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

[0 1 2]
[2 1 0]
1) positive 0.7172
2) neutral 0.2652
3) negative 0.0176


In [5]:
def analysis(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model_eng(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    if (ranking[2] == 2):
        return 0
    elif (ranking[2] == 1):
        return 1
    else:
        return -1

def analysis_vie(text):
    text = preprocess(text)
    encoded_input = torch.tensor([tokenizer_vie.encode(text)])

    with torch.no_grad():
        out = model_vie(encoded_input)
        ranking = out.logits.softmax(dim=-1).tolist()[0]
        top = np.argmax(ranking)
    if (top == 2):
        return 0
    elif (top == 1):
        return 1
    else:
        return -1


In [6]:

with open('comments_ede4_16t1kU.json', 'r') as comments:
    data = json.load(comments)
total_score = 0
negatives = 0
positives = 0
neutrals = 0
for i in data:
    comment = next(iter(i.values()))
    score = analysis(comment)
    total_score += score
    if (score == 1):
        positives+=1
    elif(score == 0):
        neutrals+=1
    else:
        negatives+=1

print(score)
print(positives)
print(neutrals)
print(negatives)

FileNotFoundError: [Errno 2] No such file or directory: 'comments_ede4_16t1kU.json'

In [ ]:
#Vietnamese Model
tokenizer_vie = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
config_vie = AutoConfig.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
model_vie = AutoModelForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")


In [ ]:
#Testing
text = "Tớ thích cậu"
print(analysis_vie(text))

1


In [ ]:
#Open a Vietnamese comments
with open("./comments/comments_CBMK6jHsR5Y.json", "r", encoding='utf-8') as comments:
    data_str = comments.read()
    data = json.loads(data_str)
#Total Score = negatives(-1) + neutrals(0) + positives(1)
total_score = 0
negatives = 0
positives = 0
neutrals = 0
for i in data:
    comment = next(iter(i.values()))
    print("- ", comment)
    score = analysis_vie(comment)
    print(">> ", score)
    print("")
    if (score == 1):
        positives+=1
    elif(score == 0):
        neutrals+=1
    else:
        negatives+=1


print("total_score", -1*negatives + positives)
print("positives:", positives)
print("neutrals:", neutrals)
print("negatives:", negatives)

-  Đúng chất hỏa Long, biểu tượng chữ V thành 2 cái râu rồng kìa. Đẹp
>>  1

-  xe điện vf rất đẹp .mong vf ngày càng phát triển hùng mạnh
>>  1

-  Xe quá đẳng cấp,tự hào Việt Nam làm được xe như vậy
>>  -1

-  Mong chú sớm sắm được một con xe Vf7; cháu cũng từng thấy chú lúc chú đi triển lãm ô tô xanh Việt Nam và review mấy xe ô tô ở triển lãm rồi chú ah.
🇻🇳
>>  1

-  Xe đẹp, chất lượng hoàn thiện có vẻ đã khá tốt hơn những xe thế hệ mới ra mắt Vf nhỉ. Mong cho VF có 1 năm thành công hơn nữa trên mọi thị trường xe.
>>  1

-  Xe đẹp quá, Mong Vinfast ngày càng phát triển, ra được mẫu sedan mui trần nữa thì tuyệt.
>>  1

-  Không mua thì cũng đừng buông lời cay nghiệt... vifat họ cũng đã nhiều cố gắng... Chúc nhà sx nhiều thành công.
>>  1

-  Có 1 Con Hoả long độc bản được chủ xe đóng hẳn cái biển vàng. Đúng chất !
>>  1

-  Chú em này đánh giá Ôtô rất chi tiết và cuốn hút người xem (nhất là đánh giá con Civic)
>>  1

-  m chạy quen chế độ sport của e34 bắt đầu thấy chưa đủ đô rồi, th